In [ ]:
# DCGAN搭建并训练的步骤
# 1.导入数据
# 2.数据预处理（[-1,1]）
# 3.搭建G生成器模型
# 4.搭建D判别者模型
# 5.构建训练步骤
# 6.训练
# 7.保存模型
# 8.测试G模型效果

In [ ]:
# 导入相关保
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# 看一下GPU有没有利用
tf.config.list_physical_devices('GPU')


In [ ]:
# 导入数据

In [ ]:
(train_img,train_labels),(test_imgs,test_labels) = tf.keras.datasets.mnist.load_data()

In [ ]:
# 查看大小
train_img.shape

In [ ]:
# 增加维度
train_img = train_img.reshape(-1,28,28,1).astype('float32')

In [ ]:
train_img.shape

In [ ]:
train_img.max(),train_img.min()

In [ ]:
# 变成[-1,1]
train_img = (train_img - 127.5) / 127.5

In [ ]:
train_img.max(),train_img.min()

In [ ]:
# 分批次

In [ ]:
# 批次数量
BATCH_SIZE = 128

In [ ]:

train_dataset = tf.data.Dataset.from_tensor_slices(train_img).shuffle(60000).batch(BATCH_SIZE)

In [ ]:
train_dataset

In [ ]:
# 取一张图来测试
for x in train_dataset:
    plt.imshow((x[0]*  127.5 +  127.5) ,cmap='gray')
    break

In [ ]:
# 搭建G生成者网络

![](./data/model_arc.png)

In [ ]:
# 搭建G网络

In [ ]:
G_model = tf.keras.Sequential([
    
    # input 100
    # output 7  x 7 x 256
    layers.Dense(7  * 7 * 256,input_shape=(100,),use_bias=False),
    # BN
    layers.BatchNormalization(),
    # ReLU
    layers.LeakyReLU(),
    # reshape
    layers.Reshape((7,7,256)),
    
    # input : 7  x 7 x 256
    # output: 7 x 7 x 128
    layers.Conv2DTranspose(128,5,1,padding='same',use_bias=False),
    # BN
    layers.BatchNormalization(),
    # ReLU
    layers.LeakyReLU(),
    
    
    # input 7 x7 x 128
    # output 14 x 14 x 64
    layers.Conv2DTranspose(64,5,2,padding='same',use_bias=False),
    # BN
    layers.BatchNormalization(),
    # ReLU
    layers.LeakyReLU(),
    
    # input 14 x 14 x 64
    # output 28 x 28 x 1
    layers.Conv2DTranspose(1,5,2,padding='same',activation='tanh')
    
    
])

In [ ]:
G_model.summary()

In [ ]:
# 随机产生一个噪音的向量，输入G网络

In [ ]:
noise_v = tf.random.normal([1,100])

In [ ]:
generated_imgs = G_model(noise_v,training=False)

In [ ]:
plt.imshow((generated_imgs[0]*  127.5 +  127.5) ,cmap='gray')

In [ ]:
# 搭建D判别器网络

![](./data/model_arc.png)

In [ ]:
# 搭建D网络

In [ ]:
D_model = tf.keras.Sequential([

    #input 28 x 28 x 1
    # output 14 x 14 x 64
    layers.Conv2D(64,5,2,padding='same',input_shape=(28,28,1)),
    # BN
    layers.BatchNormalization(),
    # ReLU
    layers.LeakyReLU(),
    # DP
    layers.Dropout(0.3),
    
    # input 14 x 14 x 64
    # output 7 x 7 x 128
    layers.Conv2D(128,5,2,padding='same'),
    # BN
    layers.BatchNormalization(),
    # ReLU
    layers.LeakyReLU(),
    # DP
    layers.Dropout(0.3),
    
    # Flatten
    layers.Flatten(),
    
    # Dense
    # input 6272
    # output 1
    layers.Dense(1)
    
])

In [ ]:
D_model.summary()

In [ ]:
# 将生成的图片输入D，查看输出
output = D_model(generated_imgs,training=False)

In [ ]:
output

In [ ]:
# 定义Loss和优化器
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
# 定义G的优化器
G_optimizer = tf.keras.optimizers.Adam(0.0001)
# 定义D的优化器
D_optimizer = tf.keras.optimizers.Adam(0.0001)

In [ ]:
# 构造训练步骤

In [ ]:
# 训练判别器：
    # 1.真实图片尽量接近1
    # 2.伪造的图片尽量接近0

# 训练生成器：伪造的图片骗过判别器（伪造的图片被判别器识别为1）

In [ ]:
@tf.function
def train_step(batch_imgs):
    
    # 训练判别器：1.真实图片尽量接近1
    
    # 为了更好记录计算，使用梯度带
    with tf.GradientTape() as tape:
        # 真实图片前传
        real_output = D_model(batch_imgs,training=True)
        ones_labels = tf.ones_like(real_output)
        # 计算一下LOSS
        d_loss = cross_entropy(ones_labels,real_output)
        
    # 计算梯度
    d_gradient = tape.gradient(d_loss,D_model.trainable_variables)
    # 优化器优化
    D_optimizer.apply_gradients(zip(d_gradient,D_model.trainable_variables))
    
    # 训练判别器：2.伪造的图片尽量接近0
    with tf.GradientTape() as tape:
        # 随机产生一个向量
        noise_v = tf.random.normal((BATCH_SIZE,100))
        # 伪造一张图
        fake_img = G_model(noise_v,training=True)
        
        # 伪造的图输入D，查看输出
        fake_output = D_model(fake_img,training=True)
        
        # 构造全0 的label
        zeros_labels = tf.zeros_like(fake_output)
        
        # 计算loss
        d_loss = cross_entropy(zeros_labels,fake_output)
        
    # 计算梯度
    d_gradient = tape.gradient(d_loss,D_model.trainable_variables)
    # 反向传播优化
    D_optimizer.apply_gradients(zip(d_gradient,D_model.trainable_variables))
    
    # 训练生成器：伪造的图片骗过判别器（伪造的图片被判别器识别为1）
    
    with tf.GradientTape() as tape:
        # 随机产生一个向量
        noise_v = tf.random.normal((BATCH_SIZE,100))
        # 伪造图片
        fake_img = G_model(noise_v,training=True)
        
        # 输入判别网络D
        fake_out =D_model(fake_img,training=True)
        
        # 构建全1 labels
        ones_labels = tf.ones_like(fake_out)
        
        # 计算一下loss
        g_loss = cross_entropy(ones_labels,fake_out)
        
    # 计算梯度
    g_gradient = tape.gradient(g_loss,G_model.trainable_variables)
    # 反向传播优化
    G_optimizer.apply_gradients(zip(g_gradient,G_model.trainable_variables))
        

In [ ]:
# 开始训练

In [ ]:
EPOCH_NUM = 100

In [ ]:
# 构建一个随机向量
noise_v = tf.random.normal((16,100))

In [ ]:
import time

In [ ]:
for epoch in range(EPOCH_NUM):
    # 记录epoch 开始训练时间
    start_time = time.time()
    # 取数据
    for batch_imgs in train_dataset:
        # 进行训练
        train_step(batch_imgs)
        
        
    print('第{}个Epoch 执行完毕，耗时{}s'.format(epoch,time.time()-start_time))
    
    # 查看一个epoch 结束后G网络表现
    generated_imgs = G_model(noise_v,training=False)
    
    # 显示这16张图
    fig = plt.figure(figsize=(4,4))
    for i in range(16):
        plt.subplot(4,4,i+1)
        plt.imshow((generated_imgs[i]*  127.5 +  127.5) ,cmap='gray')
        plt.axis('off')
    plt.show()